# Predicting Fires in Charlottesville
Jackson Barkstrom, under the guidance of Michael Lee

Initial authors: Jackson Barkstrom, Habib Karaky, Josh Schuck, Garrett Vercoe. The data we took from GitHub was worked on by many, including the four of us, during Charlottesville Civic Innovation Day 2018 (special shoutouts to Stephen and Katharine). Edited again by Jackson as of 11/9/2018 to properly add fires to housing data and to (if I have time) give a proper train test split for time. The Machine Learning section is all for the Applied Math Directed Reading Program at Brown University--work by Jackson Barkstrom under the guidance of grad student Michael Lee.

In [1]:
import pandas as pd
import numpy as np
import numpy as nd
import urllib.request, json
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Imputer, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from __future__ import print_function

/Users/jwbarkstrom/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Import Data

In [2]:
# Import data from Civic Innovation Day, and get them all in DataFrame format
fire_late = pd.read_excel('https://github.com/Smart-Cville/CID-2018-CFD-Challenge/blob/master/data/CFD_Fires_12_16to9_19_18.xlsx?raw=true')
commercial_original = pd.read_csv('https://raw.githubusercontent.com/Smart-Cville/CID-2018-CFD-Challenge/master/data/addresses-joined-to-details-commercial-with-lat-lon.csv')
residential_original = pd.read_csv('https://raw.githubusercontent.com/Smart-Cville/CID-2018-CFD-Challenge/master/data/addresses-joined-to-details-residential-with-lat-lon.csv')

In [4]:
# reading the JSON data using json.load() for the early fire data
with urllib.request.urlopen('https://github.com/Smart-Cville/CID-2018-CFD-Challenge/blob/master/data/HISTORICAL_FIRE.json?raw=true') as url:
    file = json.loads(url.read().decode())
    
datetime = []
for i in range(len(file["features"])):
    datetime.append(file['features'][i]['properties']['IN_AlarmDa'])
address = []
for i in range(len(file["features"])):
    address.append(file['features'][i]['properties']['Match_addr'])
cause = []
for i in range(len(file["features"])):
    cause.append(file['features'][i]['properties']['FR_CauseOf'])

fire_early = pd.DataFrame({'datetime':datetime, 'address':address, 'cause':cause})
del file

In [5]:
commercial = pd.DataFrame(commercial_original)
residential = pd.DataFrame(residential_original)

## Clean Data

In [6]:
# convert fire_early date column to datetime
fire_early.datetime = pd.to_datetime(fire_early.datetime)

print(fire_early.dtypes)

fire_early.head()

# perfect

datetime    datetime64[ns]
address             object
cause               object
dtype: object


,datetime,address,cause
0,2003-01-08,1002 POPLAR ST,0
1,2002-11-07,905 FOREST ST,1
2,2002-11-14,219 5TH ST SW,NULL
3,2002-12-17,822 HARDY DR,1
4,2003-01-09,1526 TRAILRIDGE RD,5


In [7]:
# select the columns we want from fire_late and format them properly

# Select only incident type and full address columns
fire_late_cleaned = pd.DataFrame(fire_late.iloc[:,[1,6,13]])

# take only numerical values from subcategory column
fire_late_cleaned.iloc[:,1] = fire_late_cleaned.iloc[:,1].str.replace("[^0-9]", "")

fire_late_cleaned.rename(columns={fire_late_cleaned.columns[1]: "cause", fire_late_cleaned.columns[0]: "date", fire_late_cleaned.columns[2]: "address"}, inplace = True)

print(fire_late_cleaned.dtypes)

# we won't add a column of 1's this time becuase left merge is too challenging for this dataset
fire_late_cleaned.head()

#note: st and rd versus street and road will be a big problem--going to have to correct here. 
# I noticed that some instances in the fire data don't have southwest, but some do, so we're going to have
# to deal with imperfect matching here. I work below to fix this.

date       datetime64[ns]
cause              object
address            object
dtype: object


,date,cause,address
0,2016-12-14,15,1155 5TH Street Southwest
1,2016-12-15,11,517 RIDGE Street
2,2016-12-20,16,10 UNIVERSITY Circle
3,2016-12-20,14,1200 5TH Street
4,2016-12-21,14,1000 1ST Street South


In [8]:
# Merge the different fields of address to get one address (so that we can merge on address)

residential['address'] = residential['st_number'].astype(str) + " " + residential['st_name'] + " " + residential['suffix']
commercial['address'] = commercial['st_number'].astype(str) + " " + commercial['st_name'] + " " + commercial['suffix']

In [9]:
# drop unused columns (we don't use a LOT of columns)
residential.drop(['parcel_number','bin','objectid','st_number','st_unit','st_name','suffix', 'predir','postdir', 'unit_type','zip', 'last_edited_user', 'last_edited_date', 'master_address_id'], 1, inplace= True)
commercial.drop(['parcel_number','bin','objectid','st_number','st_unit','st_name','suffix', 'predir','postdir', 'unit_type','zip', 'last_edited_user', 'last_edited_date', 'master_address_id'], 1, inplace= True)

In [10]:
# Turn style (# of stories) column into only digits (instead of "2 stories" it says "2")
residential['style'] = residential['style'].str.extract('(^\d*\.*\d*)', expand = True)

In [11]:
# Label respective properties with their appropriate title
residential['type'] = 'Residential'
commercial['type'] = 'Commercial'

In [12]:
commercial.head()
#commercial = commercial.drop("Unnamed: 0", 1)

,lat,lon,use_type,use_code,year_built,gross_area,story_height,number_of_stories,address,type
0,38.030637,-78.470637,R,Elementary Sch (Entire),1930.0,815.0,12.0,1.0,402 11TH ST,Commercial
1,NaN,NaN,C,Commercial,1963.0,1134.0,12.0,1.0,243 ZAN RD,Commercial
2,NaN,NaN,R,Mini-warehouse,2000.0,400.0,12.0,1.0,117 NORTH BAKER ST,Commercial
3,NaN,NaN,R,Mini-warehouse,2000.0,21600.0,18.0,2.0,702 WEST ST,Commercial
4,38.034061,-78.489876,R,Mini-warehouse,2000.0,22376.0,18.0,2.0,302 8TH ST,Commercial


In [13]:
residential.head()

,lat,lon,use_type,use_code,style,grade,ext_walls,roof,flooring,bsmt_type,...,total_rooms,bedrooms,half_bathrooms,full_bathrooms,basement_garage,sq_footage_finished_living,basement,finished_basement,address,type
0,38.064309,-78.488149,C,Duplex,2,B,Aluminum,Shingles,Other,Full Basement,...,9.0,5.0,1.0,3.0,0.0,1388.0,694.0,694.0,243 ZAN RD,Residential
1,38.027193,-78.503804,R,Single Family,1,C,Vinyl,Shingles,Hardwood,Full Basement,...,5.0,2.0,0.0,2.0,0.0,1020.0,1020.0,500.0,117 NORTH BAKER ST,Residential
2,38.034770,-78.487978,R,Single Family,1,C,Vinyl,Shingles,Hardwood,Cellar,...,5.0,3.0,0.0,1.0,0.0,925.0,250.0,0.0,702 WEST ST,Residential
3,38.039464,-78.500183,R,Single Family,1.5,C,Stucco,Shingles,Hardwood,No Basement,...,8.0,5.0,0.0,1.0,0.0,1491.0,NaN,NaN,418 17TH ST,Residential
4,-90.000000,-152.644271,R,Single Family-1 Conversion,1,C,Wood,Shingles,Hardwood,Full Basement,...,7.0,3.0,0.0,2.0,0.0,878.0,878.0,439.0,402 11TH ST,Residential


### Making Addresses Congruent (this is a simple, incomplete fix--some addresses have discrepencies such as "15th st sw" vs "15th st" that still need to be fixed)

This just uses a dictionary to change all of the fire_late_cleaned addresses to abbreviations, like in the rest of the data. We make everything lowercase first, then do the dictionary. Then we merge.

In [14]:
# make everything lowercase
commercial.address = commercial.address.str.lower()
residential.address = residential.address.str.lower()
fire_late_cleaned.address = fire_late_cleaned.address.str.lower()
fire_early.address = fire_early.address.str.lower()

In [15]:
# define dictionary and apply it to all addresses for consistency
translations = {'drive': 'dr', 'street': 'st', 'road': 'rd', 'avenue': 'ave', 'south' : 's', 'southwest': 'sw', 
                'north': 'n', 'east': 'e', 'west': 'w', 'northeast':'ne', 'northwest': 'nw', 'southeast':'se', 
                'circle':'cir', 'court': 'ct', 'lane':'ln', 'crossing': 'xing',
               'vista':'vis', 'alley':'aly', 'anex': 'anx', 'arcade': 'arc', 'bend': 'bnd', 'bluff':'blf', 
               'bluffs': 'blfs', 'bottom': 'btm', 'boulevard': 'blvd','branch':'br', 'bridge':'brg', 'brook':'brk',
               'bypass':'byp', 'canyon':'cyn', 'cape':'cpe', 'causeway':'cswy','center':'ctr', 'cliff':'clf', 
               'cliffs':'clfs', 'common':'cmn', 'commons':'cmns', 'corner':'cor', 'course':'crse','cove': 'cv',
               'crescent':'cres', 'crest':'crest', 'crossroad':'xrd', 'crossroads':'xrds', 'estate':'est', 
               'expressway':'expy', 'extension':'ext','field':'fld','flats':'flts', 'forest': 'frst', 
                'forge':'frg', 'fork':'frk', 'fort':'ft', 'freeway':'fwy', 'garden':'gdn', 'gateway':'gtwy',
               'glen':'gln', 'glens':'glns', 'green':'grn', 'grove':'grv', 'groves':'grvs', 'heights': 'hts', 
               'haven':'hvn', 'highway':'hwy', 'hill':'hl', 'hills':'hls', 'hollow':'holw', 'junction':'jct',
               'key': 'key', 'knoll': 'knl', 'lake':'lk', 'lakes':'lks', 'meadow':'mdw', 'meadows':'mdws',
               'mill':'ml', 'mount':'mt', 'mountain':'mtn', 'neck':'nck', 'parkway':'pkwy', 'passage':'psge',
                'place':'pl', 'plain':'pln', 'plains':'plns', 'plaza':'plz', 'point':'pt', 'points':'pts',
                'ridge':'rdg', 'ridges':'rdgs', 'roads':'rds', 'route':'rte', 'spring':'spg', 'springs':'spgs',
                'square': 'sq', 'station':'sta', 'streets':'sts', 'terrace':'ter', 'throughway':'trwy',
                'trace':'trc', 'track':'trk', 'trail':'trl', 'turnpike':'tpke', 'underpass':'upas', 'valley':'vly',
                'view':'vw', 'village':'vlg'}

In [16]:
# apply the dictionary to all addresses (looks pretty good for the most part)
# also removes leading and trailing whitespace
residential.address = residential.address.replace(translations, regex=True).str.strip()
commercial.address = commercial.address.replace(translations, regex=True).str.strip()
fire_late_cleaned.address = fire_late_cleaned.address.replace(translations, regex=True).str.strip()
fire_early.address = fire_early.address.replace(translations, regex=True).str.strip()

## Merge Fire Data to Commercial and Residential Data, one way for Training and another way for Testing

This is soley based on addresses, and we will do an exact match. In this case we are trying to see if past fire data can predict future fire data. For a true "train" dataset we would need to take a column with fires before 2014, and train on 2014-2016 fires, and then for a "test" have a column with fires before 2016, and a column for firest after 2016. Or something else like that.

To start with we're just seeing if we can get any insights at all (since the project is about neural nets). One column for fires before 2016 (old data), one column for latest fire (old data), and one column for if there was a fire after 2016 (new data.

First we will do the older data.

In [17]:
# this gives us what we need for one of the columns--number of fires before 2016
fire_early.address.value_counts().head()

1215 lee st           29
500 1st st s #201     17
1801 hydraulic rd     16
1600 emmet st n       10
1600 6th st se #23    10
Name: address, dtype: int64

In [18]:
# this gives us what we need for the other column--latest recorded fire
fire_early.groupby('address').datetime.max().head()

address
1 morton dr #100       2017-01-30
1 university cir       2008-03-15
10 university cir #1   2016-12-20
100 alderman rd        2015-01-02
100 e main st          2008-08-18
Name: datetime, dtype: datetime64[ns]

In [19]:
# first give every address a column for counts of fires before new fire data
final_commercial = commercial.merge(pd.DataFrame(fire_early.address.value_counts()).rename(columns = {'address': 'fire_counts'}), 
                 left_on= 'address',right_index = True, how='left')
final_residential = residential.merge(pd.DataFrame(fire_early.address.value_counts()).rename(columns = {'address': 'fire_counts'}), 
                 left_on= 'address',right_index = True, how='left')
# fills na values with 0
final_commercial.fire_counts = final_commercial.fire_counts.fillna(0).astype(np.int8)
final_residential.fire_counts = final_residential.fire_counts.fillna(0).astype(np.int8)

In [20]:
# next give every address a column for latest fire before new fire data
# this is more because I think it might be useful, will probably with feature engineering later
final_commercial = final_commercial.merge(pd.DataFrame(fire_early.groupby('address').datetime.max()).rename(columns = {'datetime': 'latest_fire'}), 
                 left_on= 'address',right_index = True, how='left')
final_residential = final_residential.merge(pd.DataFrame(fire_early.groupby('address').datetime.max()).rename(columns = {'datetime': 'latest_fire'}), 
                 left_on= 'address',right_index = True, how='left')

In [21]:
# finally, give every address a column for number of fires in the new fire data.
final_commercial = final_commercial.merge(pd.DataFrame(fire_late_cleaned.address.value_counts()).rename(columns = {'address': 'fire_result'}), 
                 left_on= 'address',right_index = True, how='left')
final_residential = final_residential.merge(pd.DataFrame(fire_late_cleaned.address.value_counts()).rename(columns = {'address': 'fire_result'}), 
                 left_on= 'address',right_index = True, how='left')
final_commercial.fire_result = final_commercial.fire_result.fillna(0).astype(np.int8)
final_residential.fire_result = final_residential.fire_result.fillna(0).astype(np.int8)

## Final Preparation for Machine Learning

In [22]:
# Get rid of all columns where use_code displays "Vacant Land"--this is just not what we're looking for
commercial_prepared   = final_commercial[final_commercial.use_code.str.contains("Vacant Land") == False]
residential_prepared = final_residential[final_residential.use_code.str.contains("Vacant Land") == False]

Obviously, everything below can be modified a lot. 

One-hot encode and column normalize via pipeline, and don't use the less meaningful columns

### Pipeline to Normalize or One-Hot Encode for Meangful Columns

The discription below is for the columns of the residential dataset but the commercial dataset pretty much follows from this. Note that we have to figure out what to do with NaN values--they're everywhere! 

Stuff below just for reference. Didn't get around to using the information much, but dropped some columns bc of it.


1) use_type (4 categories if we include nan)
2) use_code: lots of specific values and not sure if they're useful
2) style (7 categories if we merge '' with nan). Not sure what the numbers mean so should one hot?
3) grade (not really sure what this means or if it's relevant at all)
4) ext_walls: might be very helpful, should one hot, lots of columns but should be worth it
5) flooring: same as above but less columns
6) bsmt_type: extremely important, 8 categories including nan
7) heating: may be helpful? a few different types
8) fireplace: min 0 max 10, also a nan value
9) year_built: extremely important. one value says 0 (not possible) so need to give a nan for 0 and nan. should normalize so smallest 0 largest 1? (upper bound to 2018, lower to 1700): lowest in this dataset is 1730, highest 2018.
10) no_of_stories: min 0 max 3.75, should do a spectrum since it looks like we have all kinds of fractions here? although one hot is preferable I think we can get away with a spectrum. special place for nan?
11) total_rooms: extreme outlier has 705, but other than that all below 23. there is a 0 and a nan, and I will treat those seperately since I don't know what they mean. I guess you can have a structure w/o a room?
12) bedrooms: goes from 0 to 18 and nan
13) half_bathrooms: 0 to 5 and nan. not very useful and can probably drop?
14) full_bathrooms: 0 to 10 and nan. not that useful?
15) basement_garage: 0 to 3 and nan, and then a few big values in the n00s. not sure what they mean
16) sq_footage_finished_living: hugely important. max of ~8200, some nan and 0.
17) basement: hugely important. square footage of the basement, I think, max of 5290 min 0 and nan.
18) finished_basement is square footage of finished basement (so can make an unfinished_basement column with difference b/w basement and this). Max of 3790 and min 0 and nan.
19) fire_counts: numbner of past fires that have occured, normalize based on max. 

In [23]:
# convert all the category values to numerical values for the residential data (so we can one-hot later)
# note that NaN is given a value of -1, and we want to preserve it as a category so we add 1 to everything
residential_prepared.use_type  = residential_prepared.use_type.astype("category").cat.codes + 1
residential_prepared.use_code  = residential_prepared.use_code.astype("category").cat.codes + 1
residential_prepared['style']     = residential_prepared['style'].astype("category").cat.codes + 1
residential_prepared.grade     = residential_prepared.grade.astype("category").cat.codes + 1
residential_prepared.ext_walls = residential_prepared.ext_walls.astype("category").cat.codes + 1
residential_prepared.flooring  = residential_prepared.flooring.astype("category").cat.codes + 1
residential_prepared.bsmt_type = residential_prepared.bsmt_type.astype("category").cat.codes + 1
residential_prepared.heating   = residential_prepared.heating.astype("category").cat.codes + 1

/Users/jwbarkstrom/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/jwbarkstrom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [24]:
# convert all the category values to numerical values for the commercial data (so we can one-hot later)
# note that NaN is given a value of -1, and we want to preserve it as a category so we add 1 to everything
commercial_prepared.use_type = commercial_prepared.use_type.astype("category").cat.codes + 1
commercial_prepared.use_code = commercial_prepared.use_code.astype("category").cat.codes + 1

In [25]:
residential_cat = residential_prepared[["use_type", "use_code", "style", "grade", "ext_walls", "flooring", "bsmt_type", "heating"]]
residential_num = residential_prepared[["year_built", "no_of_stories", "total_rooms", "bedrooms", "half_bathrooms", "full_bathrooms", "basement_garage", "sq_footage_finished_living", "basement", "finished_basement", "fire_counts"]]

In [26]:
commercial_cat = commercial_prepared[["use_type", "use_code"]]
commercial_num = commercial_prepared[["year_built", "gross_area", "story_height", "number_of_stories", "fire_counts"]]

In [27]:
# define class DataFrameSelector in order to use our pipeline

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y = None):
        return self # nothing else to do here, just have to have a fit to fit with skleard
    def transform(self, X):
        return X[self.attribute_names].values

In [28]:
# run pipeline on commercial data
num_attribs = list(commercial_num)
cat_attribs = list(commercial_cat)

In [29]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy = "median")),
    # note: will need to mess around with this strategy and see how it affects results
    # can work on attribs_adder
    # ('attribs_adder', CombinedAttributesAdder()),
    # we need to use something like MinMaxScaler because it outputs values b/w 0 and 1
    ('std_scaler', MinMaxScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    # need to import this from future encoders!! how to do that I don't know!! There's a future_encoders.py
    # file on the jupyter notebook for the textbook
    ('cat_encoder', OneHotEncoder(sparse = False)),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [30]:
commercial_ready_x = full_pipeline.fit_transform(commercial_prepared)

In [31]:
# run pipeline on residential data 
num_attribs = list(residential_num)
cat_attribs = list(residential_cat)

In [32]:
# cut and pasted from above so it's easier to run the code
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy = "median")),
    # note: will need to mess around with this strategy and see how it affects results
    # can work on attribs_adder
    # ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    # need to import this from future encoders!! how to do that I don't know!! There's a future_encoders.py
    # file on the jupyter notebook for the textbook
    ('cat_encoder', OneHotEncoder(sparse = False)),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [33]:
residential_ready_x = full_pipeline.fit_transform(residential_prepared)

### Train Test Split 

Eventually, when we do this correctly, we will not have to train test split here because we will have already done so in our merge.

In [34]:
# we take y to be whether or not there was a fire in the future (1 in 1 column if yes, otherwise 1 in other column)
commercial_x  = commercial_ready_x
commercial_y  = list(zip((commercial_prepared['fire_result'] > 0).astype(int), (commercial_prepared['fire_result'] == 0).astype(int)))
residential_x = residential_ready_x
residential_y = list(zip((residential_prepared['fire_result'] > 0).astype(int), (residential_prepared['fire_result'] == 0).astype(int)))

In [35]:
commercial_train_x, commercial_test_x, commercial_train_y, commercial_test_y = \
train_test_split(commercial_x, commercial_y, test_size = 0.2)

In [36]:
residential_train_x, residential_test_x, residential_train_y, residential_test_y = \
train_test_split(residential_x, residential_y, test_size = 0.2)

In [37]:
# we have 1,854 addresses, with each address represented as 82 columns
commercial_train_x.shape

(1854, 82)

## Machine Learning

### Neural Net
I use code where one can see the mathematics behind everything, with inspiration from the Gori textbook. I can't imagine it's ever going to be that "sure" of a fire, but we'll see. Starting with commercial data (smaller).

I reference http://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/, https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428

In [112]:
EPOCHS = 1000

# placeholder to switch b/w batch sizes commented out
BATCH_SIZE = 1000 # tf.placeholder(tf.int64)

# set input/output variables
x = tf.placeholder(tf.float32, [None, 153]) #datapoints for commercial are 82
y = tf.placeholder(tf.float32, [None, 2]) # 2, one for no fire and one for fire
# make them compatible with train and test
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(BATCH_SIZE).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(BATCH_SIZE).repeat()

train_data = (residential_train_x, residential_train_y)
test_data = (residential_test_x, residential_test_y)

# create the iterators from the dataset
iter = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
features, labels = iter.get_next()

# initialization operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

# define the weights and their initialization
# here we go 82-->100-->1
W1 = tf.Variable(tf.random_uniform([153, 100],minval=-0.1,\
                                                      maxval=0.1))
b1 = tf.Variable(tf.random_uniform([100],minval=0,\
                                                      maxval=0.1))
W2 = tf.Variable(tf.random_uniform([100, 2],minval=-0.1,\
                                                      maxval=0.1))
b2 = tf.Variable(tf.zeros([1]))

# define relu nonlinearity, carry out forward computation
a1 = tf.matmul(x, W1) +b1             #activation of the input
x2 = tf.nn.relu(a1)      #previously x2 = tf.sigmoid(a1)
a2 = tf.matmul(x2, W2) + b2          #activation of the output

# softmax processing for output layer and cross-entropy all happen at same time
# we can essentially view the output of the softmax as percentage likelihood
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=a2))

# this works because of how we formatted our y column
correct_prediction = tf.equal(tf.argmax(a2, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# learning as gradient descent on cross-entropy error function
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [113]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [114]:
sess.run(train_init_op, feed_dict={ x: train_data[0], y: train_data[1]})
for _ in range(EPOCHS):
    sess.run(train_step, feed_dict={x: train_data[0], y: train_data[1]})
results = [[1/(1+np.exp(-q)) for q in p] for p in sess.run(a2, feed_dict={x: train_data[0], y: train_data[1]})]
print((sess.run(tf.argmax(a2, 1), feed_dict={x: train_data[0], y: train_data[1]}) == 1).sum())
sess.run(test_init_op, feed_dict={ x: train_data[0], y: train_data[1]})
print("Accuracy: %f" % sess.run(accuracy,\
         feed_dict={ x: test_data[0], y: test_data[1]}))
test_results = test_data[1]
outputted_results = [[1/(1+np.exp(-q)) for q in p] for p in sess.run(a2, feed_dict={x: test_data[0], y: test_data[1]})]

10325
Accuracy: 0.951588


In [115]:
sess.close()

In [116]:
output = [item[0] for item in outputted_results]
results = [item[0] for item in test_results]

In [117]:
output_test = pd.DataFrame({'output':output, 'results':results})

In [118]:
output_test[output_test['output'] <0.2][output_test['output'] > 0.1]['results'].mean()

/Users/jwbarkstrom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0.04596191726854892

In [119]:
output_test[output_test['output'] <0.1]['results'].mean()

0.02459016393442623

In [123]:
output_test[output_test['output'] >0.2]['results'].mean()

0.06012269938650307

### Random Forest Classifier (for comparison)

In [ ]:
# this works ok... low type 1 error (we're right about 1/3 of the time) but high type 2 error.
from sklearn.ensemble import RandomForestClassifier

# model
forest =  RandomForestClassifier()

# train (residential first)
forest.fit(residential_train_x, residential_train_y)

# predict
forest_predictions = forest.predict(residential_test_x)

y_actual = pd.Series(list(zip(*residential_test_y))[0])
y_pred = pd.Series(list(zip(*forest_predictions))[0])

from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_actual, y_pred))

confusion = pd.crosstab(y_actual, y_pred, rownames=['Actual:'], colnames = ['Predicted:'])

print(confusion)

## Export Data

In [ ]:
# Export data as CSVs 
# Note: the javascript code is just because we're using collaboratory,
# if not in collaboratory we could also use a basic pandas function to output the CSV

from IPython.display import Javascript
js_download = """
var csv = '%s';

var filename = 'results.csv';
var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
if (navigator.msSaveBlob) { // IE 10+
    navigator.msSaveBlob(blob, filename);
} else {
    var link = document.createElement("a");
    if (link.download !== undefined) { // feature detection
        // Browsers that support HTML5 download attribute
        var url = URL.createObjectURL(blob);
        link.setAttribute("href", url);
        link.setAttribute("download", filename);
        link.style.visibility = 'hidden';
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
    }
}
""" % final_commercial.to_csv(index=False).replace('\n','\\n').replace("'","\'")

Javascript(js_download)

In [ ]:
from IPython.display import Javascript
js_download = """
var csv = '%s';

var filename = 'results.csv';
var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
if (navigator.msSaveBlob) { // IE 10+
    navigator.msSaveBlob(blob, filename);
} else {
    var link = document.createElement("a");
    if (link.download !== undefined) { // feature detection
        // Browsers that support HTML5 download attribute
        var url = URL.createObjectURL(blob);
        link.setAttribute("href", url);
        link.setAttribute("download", filename);
        link.style.visibility = 'hidden';
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
    }
}
""" % final_residential.to_csv(index=False).replace('\n','\\n').replace("'","\'")

Javascript(js_download)